In [1]:
import pickle

import numpy as np

In [8]:
with open('data/3_qubit_stab_sorted.data', 'rb') as reader:
    stab_sorted = pickle.load(reader)

In [9]:
stab_matrix = np.loadtxt('data/3_qubit_matrix_sorted.csv', delimiter=',',
                         dtype=complex)

In [10]:
selection = stab_matrix[:, ::8]

In [11]:
stab_matrix[:, 560]

array([-0.5-0.j ,  0. -0.5j,  0. -0.j ,  0. -0.j ,  0. -0.j ,  0. -0.j ,
        0. -0.5j, -0.5-0.j ])

In [12]:
stab_sorted[560]

{'vector': array([[-0.5-0.j ],
        [ 0. -0.5j],
        [ 0. -0.j ],
        [ 0. -0.j ],
        [ 0. -0.j ],
        [ 0. -0.j ],
        [ 0. -0.5j],
        [-0.5-0.j ]]),
 'latex': '\\frac{1}{2}\\big{(}-|000\\rangle-i|001\\rangle-i|110\\rangle-|111\\rangle\\big{)}',
 'check_matrix': array([[1, 1, 0, 0, 1, 1],
        [0, 0, 1, 0, 1, 1],
        [0, 0, 0, 1, 1, 0]]),
 'generator_strs': ['XYZ', 'IZY', 'ZZI']}

In [13]:
stab_sorted[561]

{'vector': array([[-0. -0.j ],
        [-0. -0.j ],
        [ 0.5-0.j ],
        [-0. -0.5j],
        [-0. -0.5j],
        [ 0.5-0.j ],
        [-0. -0.j ],
        [-0. -0.j ]]),
 'latex': '\\frac{1}{2}\\big{(}|010\\rangle-i|011\\rangle-i|100\\rangle+|101\\rangle\\big{)}',
 'check_matrix': array([[1, 1, 0, 0, 1, 1],
        [0, 0, 1, 0, 1, 1],
        [0, 0, 0, 1, 1, 0]]),
 'generator_strs': ['XYZ', 'IZY', '-ZZI']}

In [14]:
n = 3
lin_dep_mat = np.loadtxt(f'data/{n}_qubit_nullspace_nice.csv', delimiter=',', dtype=complex)
updated_mat = (lin_dep_mat * np.sqrt(2)).round(3)

In [15]:
updated_mat[updated_mat == 1.414] = np.sqrt(2)

In [31]:
np.savetxt(f'data/{n}_qubit_B.csv', updated_mat, delimiter=',', fmt='%f')

------

In [2]:
n = 5
with open(f'data/{n}_qubit_subgroups.data', 'rb') as reader:
    check_matrices = pickle.load(reader)

In [4]:
def binary_matrix_rank(A):
    """
    Finds the rank of the binary matrix A by effectively converting it
    into row echelon form.

    Parameters
    ----------
    A : np.ndarray

    N.B. This method *changes* the matrix A!

    Returns
    -------
    rank : int
        The rank of matrix A.

    """

    num_rows, num_cols = A.shape
    rank = 0

    for j in range(num_cols):
        # Find the number of rows that have a 1 in the jth column
        rows = []
        for i in range(num_rows):
            if A[i, j] == 1:
                rows.append(i)

        # If the jth column has more than one 1, use row addition to
        # remove all 1s except the first one, then remove the first
        # such row and increase the rank by 1
        if len(rows) >= 1:
            for c in range(1, len(rows)):
                A[rows[c], :] = (A[rows[c], :] + A[rows[0], :]) % 2

            A = np.delete(A, rows[0], 0)
            num_rows -= 1
            rank += 1

    # For each remaining non-zero row in A, increase the rank by 1
    for row in A:
        if sum(row) > 0:
            rank += 1

    return rank

In [13]:
binary_matrix_rank(check_matrices[0][:, :n])

0

In [26]:
check_mats_sorted = sorted(check_matrices, 
                           key=lambda mat: binary_matrix_rank(mat[:, :n]))

In [27]:
check_mats_sorted[:10]

[array([[0, 1]]), array([[1, 0]]), array([[1, 1]])]

In [28]:
with open(f'data/{n}_qubit_subgroups.data', 'wb') as writer:
    pickle.dump(check_mats_sorted, writer)

In [2]:
def num_of_stab_states(n):
    return 2**n * np.prod(2**np.arange(1, n+1) + 1)

In [3]:
num_of_stab_states(5), num_of_stab_states(5) // 2**5

(2423520, 75735)

In [4]:
num_of_stab_states(6), num_of_stab_states(6) // 2**6

(315057600, 4922775)

------

In [1]:
import numpy as np
import scipy as sp
import scipy.sparse as spr

In [10]:
B_2 = spr.load_npz('data/2_qubit_B.npz')
B_2a = spr.load_npz('data/2_qubit_B_a.npz')

In [ ]:
B_2a = B_2a.toarray()
B_2 = B_2.toarray()

In [18]:
np.array_equiv(B_2a, B_2)

True

In [6]:
B_1 = spr.load_npz('data/1_qubit_B.npz')
B_1a = spr.load_npz('data/1_qubit_B_a.npz')

In [7]:
B_1a.toarray()

array([[ 1.        +0.j,  1.        +0.j,  1.        +0.j,
         1.        +0.j],
       [ 1.        +0.j, -1.        +0.j,  0.        +1.j,
         0.        -1.j],
       [-1.41421356+0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j],
       [ 0.        +0.j, -1.41421356+0.j,  0.        +0.j,
         0.        +0.j],
       [ 0.        +0.j,  0.        +0.j, -1.41421356+0.j,
         0.        +0.j],
       [ 0.        +0.j,  0.        +0.j,  0.        +0.j,
        -1.41421356+0.j]])

In [8]:
B_1.toarray()

array([[ 1.        +0.j,  1.        +0.j,  1.        +0.j,
         1.        +0.j],
       [ 1.        +0.j, -1.        +0.j,  0.        +1.j,
         0.        -1.j],
       [-1.41421356+0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j],
       [ 0.        +0.j, -1.41421356+0.j,  0.        +0.j,
         0.        +0.j],
       [ 0.        +0.j,  0.        +0.j, -1.41421356+0.j,
         0.        +0.j],
       [ 0.        +0.j,  0.        +0.j,  0.        +0.j,
        -1.41421356+0.j]])

In [8]:
nonzero_2 = []
for j in range(B_2.shape[1]):
    nonzero_2.append(B_2[:, j].nonzero()[0])

In [9]:
nonzero_2

[array([0, 1, 4], dtype=int32),
 array([2, 3, 5], dtype=int32),
 array([0, 1, 6], dtype=int32),
 array([2, 3, 7], dtype=int32),
 array([0, 1, 8], dtype=int32),
 array([2, 3, 9], dtype=int32),
 array([ 0,  1, 10], dtype=int32),
 array([ 2,  3, 11], dtype=int32),
 array([ 0,  2, 12], dtype=int32),
 array([ 1,  3, 13], dtype=int32),
 array([ 0,  2, 14], dtype=int32),
 array([ 1,  3, 15], dtype=int32),
 array([ 0,  2, 16], dtype=int32),
 array([ 1,  3, 17], dtype=int32),
 array([ 0,  2, 18], dtype=int32),
 array([ 1,  3, 19], dtype=int32),
 array([ 0,  3, 20], dtype=int32),
 array([ 1,  2, 21], dtype=int32),
 array([ 0,  3, 22], dtype=int32),
 array([ 1,  2, 23], dtype=int32),
 array([ 0,  3, 24], dtype=int32),
 array([ 1,  2, 25], dtype=int32),
 array([ 0,  3, 26], dtype=int32),
 array([ 1,  2, 27], dtype=int32),
 array([ 4,  5, 28], dtype=int32),
 array([ 6,  7, 29], dtype=int32),
 array([ 4,  5, 30], dtype=int32),
 array([ 6,  7, 31], dtype=int32),
 array([ 8,  9, 32], dtype=int32),
 ar

In [4]:
import pickle
B_data = []
with open('data/6_B_data.data', 'rb') as reader:
    try:
        while True:
            B_data.append(pickle.load(reader))
    except EOFError:
        pass

In [5]:
len(B_data)

22784

In [13]:
B_data[:50]

[(256, 0, 40, 1),
 (1152, 0, 49, 1),
 (257, 1, 41, 1),
 (1153, 2, 51, 1),
 (258, 2, 42, 1),
 (1154, 4, 53, 1),
 (259, 3, 43, 1),
 (1155, 6, 55, 1),
 (260, 4, 44, 1),
 (1156, 8, 57, 1),
 (261, 5, 45, 1),
 (1157, 10, 59, 1),
 (262, 6, 46, 1),
 (1158, 12, 61, 1),
 (263, 7, 47, 1),
 (1159, 14, 63, 1),
 (264, 16, 56, 1),
 (1160, 16, 33, 1),
 (265, 17, 57, 1),
 (1161, 18, 35, 1),
 (266, 18, 58, 1),
 (1162, 20, 37, 1),
 (267, 19, 59, 1),
 (1163, 22, 39, 1),
 (268, 20, 60, 1),
 (1164, 24, 41, 1),
 (1165, 26, 43, 1),
 (269, 21, 61, 1),
 (1166, 28, 45, 1),
 (270, 22, 62, 1),
 (1167, 30, 47, 1),
 (271, 23, 63, 1),
 (272, 8, 32, 1),
 (1168, 17, 32, 1),
 (273, 9, 33, 1),
 (1169, 19, 34, 1),
 (274, 10, 34, 1),
 (1170, 21, 36, 1),
 (275, 11, 35, 1),
 (1171, 23, 38, 1),
 (276, 12, 36, 1),
 (1172, 25, 40, 1),
 (277, 13, 37, 1),
 (1173, 27, 42, 1),
 (278, 14, 38, 1),
 (1174, 29, 44, 1),
 (279, 15, 39, 1),
 (1175, 31, 46, 1),
 (280, 24, 48, 1),
 (1176, 1, 48, 1)]